### **Laboratorio 8**

- Juan Pablo Solis
- Diego Garcia

### **Link de github**
https://github.com/JPS4321/Lab8_DL


## **Investigacioin**

### **Modelo VGG16**
**Describir la arquitectura general de VGG16 (número de capas, tipo de
convoluciones, tamaño de entrada, número de parámetros)**

El modelo VGG16 es una arquitectura de red neuronal convolucional desarrollada por el Visual Geometry Group (VGG) de la Universidad de Oxford. Está compuesta por 16 capas con pesos entrenables, de las cuales 13 son convolucionales y 3 son totalmente conectadas. Utiliza filtros de 3x3 con paso (stride) de 1 y padding de 1, junto con capas de max pooling de 2x2 después de ciertos bloques de convolución. La entrada esperada de la red es una imagen de 224x224 píxeles con tres canales (RGB). En total, el modelo tiene aproximadamente 138 millones de parámetros, lo que lo hace uno de los modelos clásicos más grandes y detallados.

**Explicar en qué dataset fue preentrenado (ImageNet) y cómo se puede
adaptar para un nuevo problema de clasificación**

Fue preentrenado en el dataset ImageNet, que contiene más de 1.2 millones de imágenes distribuidas en 1000 clases. Este entrenamiento previo le permite servir como base para tareas de transferencia de aprendizaje: se pueden congelar las capas convolucionales (que extraen características genéricas) y reemplazar o ajustar las capas finales para adaptarlo a un nuevo problema de clasificación con diferente número de clases.

### **Dataset CIFAR**

**Describir brevemente las características del dataset (número de clases,
tamaño de imágenes, conjunto de entrenamiento y prueba).**

El dataset CIFAR (Canadian Institute for Advanced Research) es un conjunto de imágenes pequeñas ampliamente utilizado para entrenar y evaluar modelos de clasificación de imágenes. Existen dos variantes comunes: CIFAR-10 y CIFAR-100. En CIFAR-10 hay 10 clases (como aviones, autos, gatos, perros, etc.), mientras que en CIFAR-100 hay 100 clases más detalladas. Cada imagen tiene un tamaño de 32x32 píxeles con 3 canales (RGB), y el conjunto de datos se divide en 50,000 imágenes para entrenamiento y 10,000 para prueba.

**Explicar cómo se deben aplicar las transformaciones y normalizaciones de
las imágenes para que sean compatibles con VGG16**

Para usar CIFAR con VGG16, es necesario redimensionar las imágenes a 224x224 píxeles, ya que ese es el tamaño de entrada esperado por el modelo. Además, se deben normalizar los valores de los píxeles utilizando las mismas estadísticas que el modelo preentrenado en ImageNet, osea restar la media [0.485, 0.456, 0.406] y dividir por la desviación estándar [0.229, 0.224, 0.225] por canal.

### **Implementacion de modelo**

#### **Evaluacion sin entrenamiento adicional**

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np


In [7]:
# Cargar dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalizar valores (0-1)
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

# Codificar etiquetas
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

print("Tamaño de entrenamiento:", x_train.shape)
print("Tamaño de prueba:", x_test.shape)


Tamaño de entrenamiento: (50000, 32, 32, 3)
Tamaño de prueba: (10000, 32, 32, 3)


In [8]:
# VGG16 sin la capa fully connected final
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.summary()


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

### **Evaluacion sin entrenamineto adicional (Parte A)**

In [9]:
# Congelar todas las capas
for layer in base_model.layers:
    layer.trainable = False

# Crear nuevo modelo con capa final para 10 clases
model_a = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

# Compilar para evaluar
model_a.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Evaluar sin entrenamiento (solo forward pass)
loss_a, acc_a = model_a.evaluate(x_test, y_test, verbose=1)

print(f"Evaluación sin entrenamiento → Loss: {loss_a:.4f}, Accuracy: {acc_a:.4f}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.1129 - loss: 2.4099
Evaluación sin entrenamiento → Loss: 2.4136, Accuracy: 0.1124


El modelo VGG16 preentrenado en ImageNet no reconoce correctamente las clases del dataset CIFAR-10, ya que nunca fue ajustado a ese dominio.
El 11 % de precisión es casi igual al azar

### **Feature Extraction (Parte B)**

In [10]:
# Congelar capas convolucionales del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Crear modelo nuevo para fine-tuning solo de la capa final
model_b = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# Compilar
model_b.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar solo la capa de clasificación
history_b = model_b.fit(
    x_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(x_test, y_test),
    verbose=1
)


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - accuracy: 0.3919 - loss: 1.7128 - val_accuracy: 0.5472 - val_loss: 1.2937
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.5363 - loss: 1.3186 - val_accuracy: 0.5673 - val_loss: 1.2259
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.5634 - loss: 1.2472 - val_accuracy: 0.5785 - val_loss: 1.1925
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.5760 - loss: 1.2132 - val_accuracy: 0.5860 - val_loss: 1.1685
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.5833 - loss: 1.1851 - val_accuracy: 0.5953 - val_loss: 1.1476
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.5954 - loss: 1.1554 - val_accuracy: 0.5962 - val_loss: 1.1448
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.5988 - loss: 1.1402 - val_accuracy: 0.5999 - val_loss: 1.1384
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.6051 - loss: 1.1288 - 

In [11]:
# Evaluar en conjunto de entrenamiento y prueba
loss_train_b, acc_train_b = model_b.evaluate(x_train, y_train, verbose=0)
loss_test_b, acc_test_b   = model_b.evaluate(x_test,  y_test,  verbose=0)

print(f"Entrenamiento → Loss: {loss_train_b:.4f}, Accuracy: {acc_train_b:.4f}")
print(f"Prueba → Loss: {loss_test_b:.4f}, Accuracy: {acc_test_b:.4f}")


Entrenamiento → Loss: 0.9802, Accuracy: 0.6576
Prueba → Loss: 1.1187, Accuracy: 0.6084


En este caso el modelo sí aprendió a mapear las características convolucionales de VGG16 (que permanecen congeladas) hacia las clases de CIFAR-10 a través de la nueva capa densa.
El aumento de accuracy a un 60% demuestra que las features extraídas de ImageNet son reutilizables para tareas nuevas con algo de entrenamiento.

### **Fine Tuning**

In [12]:
# Descongelar todas las capas del modelo base
for layer in base_model.layers:
    layer.trainable = True

# Crear modelo completo reutilizando estructura anterior
model_c = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# Mostrar capas entrenables
print("Capas entrenables:", len(model_c.trainable_weights))


Capas entrenables: 30


In [13]:
# Compilar con tasa de aprendizaje pequeña
model_c.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_c.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,586 (56.64 MB)

 Trainable params: 14,848,586 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Entrenar el modelo completo (fine-tuning)
history_c = model_c.fit(
    x_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(x_test, y_test),
    verbose=1
)


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 70ms/step - accuracy: 0.4278 - loss: 1.6104 - val_accuracy: 0.7252 - val_loss: 0.7924
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 62ms/step - accuracy: 0.7208 - loss: 0.8332 - val_accuracy: 0.7685 - val_loss: 0.6769
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 62ms/step - accuracy: 0.7715 - loss: 0.6685 - val_accuracy: 0.7894 - val_loss: 0.6109
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 62ms/step - accuracy: 0.8096 - loss: 0.5672 - val_accuracy: 0.8052 - val_loss: 0.5812
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 62ms/step - accuracy: 0.8334 - loss: 0.4952 - val_accuracy: 0.8173 - val_loss: 0.5357
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 62ms/step - accuracy: 0.8554 - loss: 0.4342 - val_accuracy: 0.8224 - val_loss: 0.5232
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 62ms/step - accuracy: 0.8743 - loss: 0.3783 - val_accuracy: 0.8278 - val_loss: 0.5157
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 63ms/step - accuracy: 0.8862 - loss: 0.3381 - 

In [15]:
# Evaluar en conjunto de entrenamiento y prueba
loss_train_c, acc_train_c = model_c.evaluate(x_train, y_train, verbose=0)
loss_test_c,  acc_test_c  = model_c.evaluate(x_test,  y_test,  verbose=0)

print(f"Entrenamiento → Loss: {loss_train_c:.4f}, Accuracy: {acc_train_c:.4f}")
print(f"Prueba → Loss: {loss_test_c:.4f}, Accuracy: {acc_test_c:.4f}")


Entrenamiento → Loss: 0.1659, Accuracy: 0.9471
Prueba → Loss: 0.5074, Accuracy: 0.8419
